# Observations and Insights
###### The first step of this assigment is to use Google Places API and create a heat map that displays the humidity for 500+ cities around the world. Use the Lat and Lng as locations and Humidity as the weight. 
###### As of 07/20/2020 we can see the top 5 cities with mayor humidity are:  
######  1. Coyhaique, CL
######  2. Airai, TL
######  3. Clyde River,	CA
######  4. Ketchikan, US
######  5. Puerto Madero, MX	
###### As of 07/20/2020 The top 5 cities with less humidity are :
######  1. Sonqor,	IR
######  2. Balad,	IQ
######  3. Kimberley,	ZA	
######  4. Kharan,	PK
######  5. Fasa, IR
###### The second step of this assigment is narrow down the cities, to fit ideal weather conditions like: A temperature between 70 and 80 degrees, wind speed less than 10 mph and Zero cloudiness. This data subset were use to call Google Places API and find the first hotel for each city located within 5000 meters of your coordinates. Next, plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.
###### We can see  the mayor number of the Hotels that fit ideal weather conditions are located in Africa, one in Indonesia, one in Rusia, one in Portugal and one in Brasil.

In [ ]:
# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [45]:
#Loading city_weather csv
weather_csv="../WeatherPy/city_weather.csv"
#Read city_weather csv file
city_weather=pd.read_csv(weather_csv)
#Data Frame
city_weather_df=pd.DataFrame(city_weather)
city_weather_df.head()

city_weather_df_sort = city_weather_df.sort_values('Humidity',ascending = False)
city_weather_df_sort

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
124,124,Coyhaique,90,CL,1595252483,100,-45.58,-72.07,33.80,3.36
205,205,Airai,37,TL,1595252503,100,-8.93,125.41,55.94,2.51
154,154,Clyde River,90,CA,1595252490,100,70.47,-68.59,41.00,8.05
433,433,Ketchikan,90,US,1595252490,100,55.34,-131.65,55.40,6.93
432,432,Puerto Madero,75,MX,1595252355,100,14.72,-92.42,77.00,2.91
...,...,...,...,...,...,...,...,...,...,...
176,176,Tabas,0,IR,1595252496,8,33.60,56.92,105.80,6.93
564,564,Kimberley,0,ZA,1595252577,7,-28.73,24.76,75.20,4.70
504,504,Kharan,0,PK,1595252564,7,28.58,65.42,103.98,14.29
572,572,Balad,0,IQ,1595252579,5,34.02,44.15,115.45,23.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [40]:
locations = city_weather_df[["Lat", "Lng"]].astype(float)
humidity_rate = city_weather_df["Humidity"].astype(float)


In [26]:
#Use the Lat and Lng as locations and Humidity as the weight.
# Create a Humidity Heatmap layer
fig = gmaps.figure(center=(40.0,0.0),zoom_level=2.5)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
# Suggested:A max temperature lower than 80 degrees but higher than 70. 
maxtemperature=city_weather_df.loc[city_weather_df["Max Temp"]<80,:]
mintemperature=maxtemperature.loc[maxtemperature["Max Temp"]>70,:]

#Suggested: Wind speed less than 10 mph.
wind_speed=mintemperature.loc[mintemperature["Wind Speed"]<10,:]
# Zero cloudiness.
zero_cloudiness=wind_speed.loc[wind_speed["Cloudiness"]==0,:]
zero_cloudiness_df1=pd.DataFrame(zero_cloudiness)

# Drop any rows that don't contain all three conditions. 
#You want to be sure the weather is ideal
zero_cloudiness_df=zero_cloudiness_df1.dropna(axis=0, how='any')

zero_cloudiness_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
38,38,Somerset East,0,ZA,1595252465,17,-32.72,25.59,74.79,9.73
52,52,Cidreira,0,BR,1595252468,70,-30.18,-50.21,71.15,2.91
70,70,Cape Town,0,ZA,1595252472,52,-33.93,18.42,71.01,6.93
98,98,Sinazongwe,0,ZM,1595252478,35,-17.26,27.46,75.49,7.16
117,117,Karema,0,TZ,1595252482,49,-6.82,30.44,77.72,8.14
120,120,East London,0,ZA,1595252482,17,-33.02,27.91,75.20,5.82
168,168,Shakawe,0,BW,1595252494,17,-18.37,21.85,78.78,8.03
196,196,Ponta do Sol,0,PT,1595252501,62,32.67,-17.10,73.00,0.87
302,302,Saldanha,0,ZA,1595252314,53,-33.01,17.94,73.40,3.36
337,337,Ondjiva,0,AO,1595252531,18,-17.07,15.73,79.99,9.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df=pd.DataFrame(zero_cloudiness_df)
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
38,38,Somerset East,0,ZA,1595252465,17,-32.72,25.59,74.79,9.73,
52,52,Cidreira,0,BR,1595252468,70,-30.18,-50.21,71.15,2.91,
70,70,Cape Town,0,ZA,1595252472,52,-33.93,18.42,71.01,6.93,
98,98,Sinazongwe,0,ZM,1595252478,35,-17.26,27.46,75.49,7.16,
117,117,Karema,0,TZ,1595252482,49,-6.82,30.44,77.72,8.14,
120,120,East London,0,ZA,1595252482,17,-33.02,27.91,75.20,5.82,
168,168,Shakawe,0,BW,1595252494,17,-18.37,21.85,78.78,8.03,
196,196,Ponta do Sol,0,PT,1595252501,62,32.67,-17.10,73.00,0.87,
302,302,Saldanha,0,ZA,1595252314,53,-33.01,17.94,73.40,3.36,
337,337,Ondjiva,0,AO,1595252531,18,-17.07,15.73,79.99,9.55,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#target_coordinates = f"{Lat},{Lng}"
#target_search = "Hotel"
target_radius = 5000
target_type = "lodging"
params = {
          "radius": target_radius,
         #"location":target_coordinates,
          "types": target_type,
          #"keyword": target_search, 
          "key": g_key
         }

In [10]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city= row["City"]
    country=row["Country"]
    target_coordinates = f"{lat},{lng}" 
    print(f"Retrieving Results for Index {index}: {city}.")
    params['location'] = target_coordinates
    #make request and print url
    hotel_name = requests.get(base_url, params=params).json()
#    print(json.dumps(hotel_name, indent=4, sort_keys=True))
#    print(hotel_name)
    # extract results
    results = hotel_name['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 38: Somerset East.
Closest hotel is Mbali Lodge.
Retrieving Results for Index 52: Cidreira.
Closest hotel is Novomar Pousada e Turismo Ltda.
Retrieving Results for Index 70: Cape Town.
Closest hotel is Southern Sun Waterfront Cape Town.
Retrieving Results for Index 98: Sinazongwe.
Closest hotel is Lakeview Lodge.
Retrieving Results for Index 117: Karema.
Missing field/result... skipping.
Retrieving Results for Index 120: East London.
Closest hotel is Tu Casa.
Retrieving Results for Index 168: Shakawe.
Closest hotel is Ditlhapi Guest House.
Retrieving Results for Index 196: Ponta do Sol.
Closest hotel is Hotel do Campo.
Retrieving Results for Index 302: Saldanha.
Closest hotel is Blue Bay Lodge.
Retrieving Results for Index 337: Ondjiva.
Closest hotel is SOS Motel.
Retrieving Results for Index 422: Pangkalanbuun.
Closest hotel is Grand Kecubung Hotel.
Retrieving Results for Index 441: Usinsk.
Closest hotel is Dobro Pozhalovaty.
Retrieving Results for Index 5

In [36]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
38,38,Somerset East,0,ZA,1595252465,17,-32.72,25.59,74.79,9.73,Mbali Lodge
52,52,Cidreira,0,BR,1595252468,70,-30.18,-50.21,71.15,2.91,Novomar Pousada e Turismo Ltda
70,70,Cape Town,0,ZA,1595252472,52,-33.93,18.42,71.01,6.93,Southern Sun Waterfront Cape Town
98,98,Sinazongwe,0,ZM,1595252478,35,-17.26,27.46,75.49,7.16,Lakeview Lodge
117,117,Karema,0,TZ,1595252482,49,-6.82,30.44,77.72,8.14,
120,120,East London,0,ZA,1595252482,17,-33.02,27.91,75.20,5.82,Tu Casa
168,168,Shakawe,0,BW,1595252494,17,-18.37,21.85,78.78,8.03,Ditlhapi Guest House
196,196,Ponta do Sol,0,PT,1595252501,62,32.67,-17.10,73.00,0.87,Hotel do Campo
302,302,Saldanha,0,ZA,1595252314,53,-33.01,17.94,73.40,3.36,Blue Bay Lodge
337,337,Ondjiva,0,AO,1595252531,18,-17.07,15.73,79.99,9.55,SOS Motel


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations,info_box_content=[f"Nearest Hotel:{hotel}"for hotel in hotel_info])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…